In [1]:
import tensorflow as tf
import numpy as np
import random
import csv
import pathlib

In [2]:
#"阿弥陀佛", "观世音菩萨", "观音菩萨", "南无阿弥陀佛", "南无观世音菩萨"

fohao_to_eng = {"阿弥陀佛":"amtf", "观世音菩萨":"gsyps", "观音菩萨":"gyps", "南无阿弥陀佛":"nmamtf", "南无观世音菩萨":"nmgsyps"}
eng_to_fohao = {"amtf":"阿弥陀佛", "gsyps":"观世音菩萨", "gyps":"观音菩萨", "nmamtf":"南无阿弥陀佛", "nmgsyps":"南无观世音菩萨"}

fohao_chinese = "阿弥陀佛"
fohao = fohao_chinese
#fohao =  = fohao_to_eng[fohao_chinese]
MODEL_SAVE_PATH = r"D:/语音识别/念佛计数/h5/"
#MODEL_SAVE_PATH = r"./"

print("=============================准备训练：{}====================================".format(fohao_chinese))
#=================================================================

def LoadCSV(path):    
    with open(path) as csvfile: 
        reader = csv.reader(csvfile)
        lx = []
        ly = []
        for row in reader:
            lx.append(row[0:-1])
            ly.append(row[-1])   
        lx = np.reshape(lx, (-1, 5, 39)).astype("float32")        
        ly = np.array(ly).astype("float32")        
        print("load csv successed!:{0} ".format(path))
    return lx, ly

lstm_seq_length = 5
dim = lstm_seq_length * 32

def getData(path):
    features, labels = LoadCSV(path)
    features_expand = np.expand_dims(features, axis=-1)
    rps = model_rec.predict(features_expand)    
    
    data_x, data_y = [], []
    data_len = len(labels) - lstm_seq_length
    for i in range(0, len(features) - lstm_seq_length):
        data_x.append(np.reshape(rps[i:i+lstm_seq_length], (dim,)))  
    data_x = np.expand_dims(data_x, axis=0)
    data_y = np.reshape(labels[0:data_len], (1,data_len,1))   
    print("create data shape x:{}, y:{}".format(data_x.shape, data_y.shape))       
    return data_x, data_y


CSV_FILE_DIR = r"D:/语音识别/念佛计数/语言模型CSV/{}/".format(fohao)
MODEL_REC_SAVEFILE_PATH = "{}ACOUSTIC_32_MODEL_{}.h5".format(r"D:/语音识别/念佛计数/h5/", fohao)
model_rec = tf.keras.models.load_model(MODEL_REC_SAVEFILE_PATH)

print("=============================创建模型====================================")
#=================================================================

lstm_seq_length = 5
dim = lstm_seq_length * 32

model_ctor = tf.keras.Sequential()
model_ctor.add(tf.keras.layers.LSTM(256, batch_input_shape=(1, None,dim), return_sequences=True, stateful=True))
model_ctor.add(tf.keras.layers.LSTM(256, return_sequences=True, stateful=True))
model_ctor.add(tf.keras.layers.Dense(1024, activation='relu'))
model_ctor.add(tf.keras.layers.Dense(512, activation='relu'))
model_ctor.add(tf.keras.layers.Dense(128, activation='relu'))
model_ctor.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model_ctor.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss='binary_crossentropy', metrics=['acc'])
print(model_ctor.summary())


=============================准备训练：阿弥陀佛====================================
=============================创建模型====================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (1, None, 256)            427008    
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 256)            525312    
_________________________________________________________________
dense (Dense)                (1, None, 1024)           263168    
_________________________________________________________________
dense_1 (Dense)              (1, None, 512)            524800    
_________________________________________________________________
dense_2 (Dense)              (1, None, 128)            65664     
_________________________________________________________________
dense_3 (Dense)              (1, None, 1)  

In [ ]:
out_epochs = 1
epochs = 100

csv_root = pathlib.Path(CSV_FILE_DIR)
filelist = list(csv_root.glob("*"))
total_epochs = 1
for oe in range(out_epochs):
    random.shuffle(filelist)
    for file in filelist:    
        path = pathlib.Path(file)     
        print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")    
        print("out_epoch:{}/{}".format(total_epochs, len(filelist) * out_epochs))    
        train_x, train_y  = getData(path)
        print((train_x.shape, train_y.shape))
        model_ctor.reset_states()
        model_ctor.fit(train_x, train_y, epochs=epochs) 
        total_epochs += 1
print("finished!")

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
out_epoch:1/13
load csv successed!:D:\语音识别\念佛计数\语言模型CSV\阿弥陀佛\000_012.csv 
create data shape x:(1, 749, 160), y:(1, 749, 1)
((1, 749, 160), (1, 749, 1))
Train on 1 samples
Epoch 1/100
1/1 [==============================] - 5s 5s/sample - loss: 0.7048 - acc: 0.4860
Epoch 2/100
1/1 [==============================] - 2s 2s/sample - loss: 0.6888 - acc: 0.4860
Epoch 3/100
1/1 [==============================] - 2s 2s/sample - loss: 0.6766 - acc: 0.7810
Epoch 4/100
1/1 [==============================] - 2s 2s/sample - loss: 0.6652 - acc: 0.8264
Epoch 5/100
1/1 [==============================] - 2s 2s/sample - loss: 0.6534 - acc: 0.8198
Epoch 6/100
1/1 [==============================] - 2s 2s/sample - loss: 0.6398 - acc: 0.8278
Epoch 7/100
1/1 [==============================] - 2s 2s/sample - loss: 0.6253 - acc: 0.8571
Epoch 8/100
1/1 [==============================] - 2s 2s/sample - loss: 0.6099 

Epoch 86/100
1/1 [==============================] - 2s 2s/sample - loss: 0.0059 - acc: 1.0000
Epoch 87/100
1/1 [==============================] - 2s 2s/sample - loss: 0.0054 - acc: 1.0000
Epoch 88/100
1/1 [==============================] - 2s 2s/sample - loss: 0.0047 - acc: 1.0000
Epoch 89/100
1/1 [==============================] - 2s 2s/sample - loss: 0.0044 - acc: 1.0000
Epoch 90/100
1/1 [==============================] - 2s 2s/sample - loss: 0.0039 - acc: 1.0000
Epoch 91/100
1/1 [==============================] - 2s 2s/sample - loss: 0.0036 - acc: 1.0000
Epoch 92/100
1/1 [==============================] - 3s 3s/sample - loss: 0.0032 - acc: 1.0000
Epoch 93/100
1/1 [==============================] - 3s 3s/sample - loss: 0.0030 - acc: 1.0000
Epoch 94/100
1/1 [==============================] - 2s 2s/sample - loss: 0.0027 - acc: 1.0000
Epoch 95/100
1/1 [==============================] - 2s 2s/sample - loss: 0.0024 - acc: 1.0000
Epoch 96/100


In [ ]:
MODEL_SAVEFILE_PATH = "{}LANG_MODEL_{}.h5".format(MODEL_SAVE_PATH, eng_to_fohao[fohao])
model_ctor.save(MODEL_SAVEFILE_PATH)
print("saved:{}".format(MODEL_SAVEFILE_PATH))